In [ ]:
import os
import tensorflow as tf
import logging
tf.get_logger().setLevel(logging.ERROR)
print("TensorFlow version:", tf.__version__)

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)

# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=20, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=20, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

In [ ]:
import tensorflow as tf

from tensorflow import keras
import keras

print(tf.__version__)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns


In [ ]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight','Acceleration', 'Model Year', 'Origin']

raw_dataset = pd.read_csv(url, names=column_names,na_values='?', comment='\t',sep=' ', skipinitialspace=True)

In [ ]:
raw_dataset

In [ ]:

dataset = raw_dataset.copy()
dataset.tail()



In [ ]:
dataset.isna().sum()

In [ ]:
dataset = dataset.dropna()

In [ ]:
dataset['Origin'] = dataset['Origin'].map({1: 'USA', 2: 'Europe', 3: 'Japan'})


In [ ]:
dataset = pd.get_dummies(dataset, columns=['Origin'], prefix='', prefix_sep='')
dataset.tail()

In [ ]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

In [ ]:
sns.pairplot(train_dataset[['MPG', 'Cylinders', 'Displacement', 'Weight']], diag_kind='kde')


In [ ]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('MPG')
test_labels = test_features.pop('MPG')

In [ ]:
train_dataset.describe().transpose()[['mean', 'std']]

In [ ]:
normalizer = keras.layers.Normalization(axis=-1)


In [ ]:
normalizer.adapt(np.array(train_features))

In [ ]:
print(normalizer.mean.numpy())


In [ ]:
first = np.array(train_features[:1])
first = np.asarray(first).astype('float32')
with tf.device('/cpu:0'):
    with np.printoptions(precision=2, suppress=True):
      print('First example:', first)
      print()
      print('Normalized:', normalizer(first).numpy())

In [ ]:
with tf.device('/cpu:0'):
    horsepower = np.array(train_features['Horsepower'])
    
    horsepower_normalizer = keras.layers.Normalization(input_shape=[1,], axis=None)
    horsepower_normalizer.adapt(horsepower)


In [ ]:
with tf.device('/cpu:0'):

    horsepower_model = keras.Sequential([
        horsepower_normalizer,
        keras.layers.Dense(units=1)
    ])
    
    horsepower_model.summary()

In [ ]:
with tf.device('/cpu:0'):
    horsepower_model.predict(horsepower[:10])

In [ ]:
with tf.device('/cpu:0'):
    horsepower_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),loss='mean_absolute_error')

In [ ]:
with tf.device('/cpu:0'):
    history = horsepower_model.fit(
        train_features['Horsepower'],
        train_labels,
        epochs=100,
        # Suppress logging.
        verbose=0,
        # Calculate validation results on 20% of the training data.
        validation_split = 0.2)

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
def plot_loss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.ylim([0, 10])
  plt.xlabel('Epoch')
  plt.ylabel('Error [MPG]')
  plt.legend()
  plt.grid(True)

In [ ]:
plot_loss(history)

In [ ]:
with tf.device('/cpu:0'):
    test_results = {}
    test_results['horsepower_model'] = horsepower_model.evaluate(test_features['Horsepower'],test_labels, verbose=0)

In [ ]:
with tf.device('/cpu:0'):
    x = tf.linspace(0.0, 250, 251)
    y = horsepower_model.predict(x)

In [ ]:
def plot_horsepower(x, y):
  plt.scatter(train_features['Horsepower'], train_labels, label='Data')
  plt.plot(x, y, color='k', label='Predictions')
  plt.xlabel('Horsepower')
  plt.ylabel('MPG')
  plt.legend()

In [ ]:
plot_horsepower(x,y)

In [ ]:
## MULTIPLE

In [ ]:
with tf.device('/cpu:0'):
    linear_model = keras.Sequential([normalizer,keras.layers.Dense(units=1)])
    linear_model.predict(train_features[:10])

In [ ]:
linear_model.layers[1].kernel

In [ ]:
with tf.device('/cpu:0'):
    linear_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.1),loss='mean_absolute_error')
    history = linear_model.fit(
        train_features,
        train_labels,
        epochs=100,
        # Suppress logging.
        verbose=0,
        # Calculate validation results on 20% of the training data.
        validation_split = 0.2)

In [ ]:
plot_loss(history)

In [ ]:
with tf.device('/cpu:0'):
    test_results['linear_model'] = linear_model.evaluate(test_features, test_labels, verbose=0)

In [ ]:
with tf.device('/cpu:0'):
    def build_and_compile_model(norm):
      model = keras.Sequential([
          norm,
          keras.layers.Dense(64, activation='relu'),
          keras.layers.Dense(64, activation='relu'),
          keras.layers.Dense(1)
      ])
    
      model.compile(loss='mean_absolute_error',
                    optimizer=keras.optimizers.Adam(0.001))
      return model
    dnn_horsepower_model = build_and_compile_model(horsepower_normalizer)

In [ ]:
dnn_horsepower_model.summary()

In [ ]:
with tf.device('/cpu:0'):
    history = dnn_horsepower_model.fit(
        train_features['Horsepower'],
        train_labels,
        validation_split=0.2,
        verbose=0, epochs=100)

In [ ]:
plot_loss(history)


In [ ]:
with tf.device('/cpu:0'):
    x = tf.linspace(0.0, 250, 251)
    y = dnn_horsepower_model.predict(x)

In [ ]:
plot_horsepower(x, y)

In [ ]:
with tf.device('/cpu:0'):
    test_results['dnn_horsepower_model'] = dnn_horsepower_model.evaluate(
        test_features['Horsepower'], test_labels,
        verbose=0)

In [ ]:
with tf.device('/cpu:0'):
    dnn_model = build_and_compile_model(normalizer)
    dnn_model.summary()

In [ ]:
with tf.device('/cpu:0'):
    history = dnn_model.fit(
        train_features,
        train_labels,
        validation_split=0.2,
        verbose=0, epochs=100)

In [ ]:
plot_loss(history)

In [ ]:
with tf.device('/cpu:0'):
    test_results['dnn_model'] = dnn_model.evaluate(test_features, test_labels, verbose=0)


In [ ]:
pd.DataFrame(test_results, index=['Mean absolute error [MPG]']).T

In [ ]:
with tf.device('/cpu:0'):
    test_predictions = dnn_model.predict(test_features).flatten()
    
    a = plt.axes(aspect='equal')
    plt.scatter(test_labels, test_predictions)
    plt.xlabel('True Values [MPG]')
    plt.ylabel('Predictions [MPG]')
    lims = [0, 50]
    plt.xlim(lims)
    plt.ylim(lims)
    _ = plt.plot(lims, lims)

In [ ]:
error = test_predictions - test_labels
plt.hist(error, bins=25)
plt.xlabel('Prediction Error [MPG]')
_ = plt.ylabel('Count')